In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # punctuation
    text = text.lower().strip()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Load datasets
go_emotions = pd.read_csv("go_emotions_dataset.csv")
emotion_69k = pd.read_csv("emotion-emotion_69k.csv")
reddit = pd.read_csv("reddit_text-davinci-002.csv")

# Apply cleaning# Apply cleaning to each dataset using the correct column names
go_emotions["clean_text"] = go_emotions["text"].astype(str).apply(clean_text)

emotion_69k["clean_text"] = emotion_69k["Situation"].astype(str).apply(clean_text)

reddit["clean_text"] = reddit["prompt"].astype(str).apply(clean_text)



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shrivarshininarayanan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
emotion_69k["emotion"] = emotion_69k["emotion"].fillna("unknown")
emotion_69k["labels"] = emotion_69k["emotion"].apply(lambda x: x.split(','))


In [3]:
print(emotion_69k["labels"].sample(5))


26932    [devastated]
47094     [surprised]
29365       [annoyed]
15267       [hopeful]
15961       [jealous]
Name: labels, dtype: object


In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset, Features, Sequence, Value
import numpy as np
import torch

# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Prepare labels
emotion_69k["labels"] = emotion_69k["emotion"].apply(lambda x: x.split(',') if isinstance(x, str) else ["unknown"])
mlb = MultiLabelBinarizer()
label_matrix = mlb.fit_transform(emotion_69k["labels"]).astype(np.float32)

# Explicitly define the dataset structure
features = Features({
    'clean_text': Value('string'),
    'labels': Sequence(Value(dtype='float32'))
})

# Build dataset with proper float32 labels
dataset = Dataset.from_dict({
    "clean_text": emotion_69k["clean_text"].tolist(),
    "labels": label_matrix.tolist()
}, features=features)

# Tokenize
def tokenize(batch):
    return tokenizer(batch["clean_text"], padding=True, truncation=True)

tokenized = dataset.map(tokenize, batched=True)

# Format for PyTorch
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Load model and setup
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=label_matrix.shape[1])
model.config.problem_type = "multi_label_classification"

# Training config
training_args = TrainingArguments(
    output_dir="./emotion_roberta",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized,
    data_collator=data_collator
)

# Train
trainer.train()
 

Map:   0%|          | 0/64636 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.045100,0.039141
2,0.029900,0.022624


TrainOutput(global_step=16160, training_loss=0.05006685788088506, metrics={'train_runtime': 51323.4191, 'train_samples_per_second': 2.519, 'train_steps_per_second': 0.315, 'total_flos': 3091861183034304.0, 'train_loss': 0.05006685788088506, 'epoch': 2.0})

In [5]:
# Evaluate performance
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)
# Save labels
import joblib
joblib.dump(mlb.classes_, "emotion_labels.pkl")


Evaluation Results: {'eval_loss': 0.022623591125011444, 'eval_runtime': 5935.2605, 'eval_samples_per_second': 10.89, 'eval_steps_per_second': 1.361, 'epoch': 2.0}


['emotion_labels.pkl']

In [6]:
from sklearn.metrics import classification_report

# Get predictions
preds_output = trainer.predict(tokenized)
pred_labels = (preds_output.predictions > 0.5).astype(int)
true_labels = np.array(preds_output.label_ids)

# Evaluation report
print(classification_report(true_labels, pred_labels, target_names=mlb.classes_))


                                                                                                                                                                                                precision    recall  f1-score   support

                                                                                                                                                                           I really killed it!       0.00      0.00      0.00         4
                                                                                                            a boy.  I hear all these different labor stories that aren't exactly reassuring!         0.00      0.00      0.00         3
 but what I didn't know was that he was working in the next room with the door open.  He approached and asked what I had been saying.  I knew I was caught.  I was so disgusted with myself.         0.00      0.00      0.00         4
                                                                       

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
model.save_pretrained("saved_roberta_emotion_model-v1")
tokenizer.save_pretrained("saved_roberta_emotion_model-v1")
# from transformers import RobertaForSequenceClassification, RobertaTokenizer

# model = RobertaForSequenceClassification.from_pretrained("saved_roberta_emotion_model")
# tokenizer = RobertaTokenizer.from_pretrained("saved_roberta_emotion_model")


('saved_roberta_emotion_model-v1/tokenizer_config.json',
 'saved_roberta_emotion_model-v1/special_tokens_map.json',
 'saved_roberta_emotion_model-v1/vocab.json',
 'saved_roberta_emotion_model-v1/merges.txt',
 'saved_roberta_emotion_model-v1/added_tokens.json')

In [ ]:
# ======================================
# NLP Mental Health Chatbot - Final Version (Based on Project Proposal)
# Emotion Detection: Fine-tuned RoBERTa
# Intent Detection: Sentence-BERT + Semantic Matching
# Response Generation: LLaMA2 via Ollama
# ======================================

import os
import json
import torch
import pandas as pd
import subprocess
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sentence_transformers import SentenceTransformer
import joblib

# Set environment to suppress warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["MallocStackLogging"] = "0"

# ============ STEP 1: Load Fine-Tuned RoBERTa Emotion Model ============
print("Loading fine-tuned RoBERTa emotion classifier...")
emotion_model = RobertaForSequenceClassification.from_pretrained("saved_roberta_emotion_model-v1")
emotion_tokenizer = RobertaTokenizer.from_pretrained("saved_roberta_emotion_model-v1")
mlb_classes = joblib.load("emotion_labels.pkl")

# ============ STEP 2: Load Sentence-BERT for Intent Detection ============
print("Loading intent patterns from KB.json...")
with open("KB.json", "r") as f:
    kb = json.load(f)

intent_patterns = []
intent_tags = []
for intent in kb["intents"]:
    tag = intent["tag"]
    for pattern in intent.get("patterns", []):
        if pattern.strip():
            intent_patterns.append(pattern.strip())
            intent_tags.append(tag)

print("Loading Sentence-BERT model for intent classification...")
intent_encoder = SentenceTransformer("all-MiniLM-L6-v2")
intent_embeddings = intent_encoder.encode(intent_patterns)

# Optional: Evaluate self-accuracy
print("Evaluating intent model self-match accuracy...")
y_true = intent_tags
y_pred = []
for pattern in intent_patterns:
    input_embedding = intent_encoder.encode([pattern])
    sims = cosine_similarity(input_embedding, intent_embeddings)[0]
    best_index = sims.argsort()[-1]
    y_pred.append(intent_tags[best_index])
accuracy = accuracy_score(y_true, y_pred)
print(f"Self-match intent accuracy: {accuracy * 100:.2f}%")

# ============ STEP 3: Define Utility Functions ============

def detect_emotion_roberta(text):
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits
    probs = torch.sigmoid(logits)[0].cpu().numpy()

    # Debug print: Show emotion probabilities and mapping
    print("Raw Emotion Probabilities:")
    for i, p in enumerate(probs):
        print(f"{mlb_classes[i]}: {p:.3f}")

    threshold = 0.35  # Lowered threshold to capture more subtle emotions
    top_labels = [mlb_classes[i] for i, p in enumerate(probs) if p > threshold]

    if not top_labels:
        top_labels = [mlb_classes[int(np.argmax(probs))]]  # fallback if no emotion passes threshold

    return top_labels  # fallback if no score > 0.5

def detect_intent(text, top_k=3):
    """Use Sentence-BERT + cosine similarity to detect closest intent."""
    input_embedding = intent_encoder.encode([text])
    sims = cosine_similarity(input_embedding, intent_embeddings)[0]
    top_indices = sims.argsort()[-top_k:][::-1]
    top_matches = [(intent_tags[i], intent_patterns[i], sims[i]) for i in top_indices]
    print("Top intent matches:")
    for tag, pattern, score in top_matches:
        print(f" - {tag}: '{pattern}' ({score:.2f})")
    return top_matches[0][0]

def llama_respond(prompt: str, model: str = "llama2"):
    """Generate a response using LLaMA2 via Ollama CLI."""
    try:
        print("Calling LLaMA2 via Ollama...")
        result = subprocess.run(
            ["ollama", "run", model, prompt],
            capture_output=True,
            text=True,
            timeout=180
        )
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            print("Ollama error:", result.stderr)
            return "I'm sorry, I encountered an issue generating a response."
    except subprocess.TimeoutExpired:
        return "The response took too long to generate. Try again."

def generate_response(user_input, history=[]):
    """Pipeline: Detect intent + emotion → generate response using KB and LLaMA2."""
    emotions = detect_emotion_roberta(user_input)
    intent = detect_intent(user_input)

    # Try rule-based KB response first
    kb_response = None
    for entry in kb["intents"]:
        if entry["tag"] == intent and entry.get("responses"):
            kb_response = entry["responses"]
            break

    # Add to history
    history.append({"input": user_input, "intent": intent, "emotions": emotions})

    print("--- Debug Info ---")
    print(f"Detected Emotion(s): {emotions}")
    print(f"Detected Intent: {intent}")

    # Generate LLaMA2 continuation even if KB exists
    prompt_history = "".join([f"User: {msg['input']}Intent: {msg['intent']}Emotion(s): {msg['emotions']}"
        for msg in history[-3:]  ])
    prompt = (f"Context:{prompt_history}"f"Now, respond empathetically and constructively to the last user message.")

    if kb_response:
        selected_response = kb_response[0] if isinstance(kb_response, list) else kb_response
        print(f"Using KB-based response: {selected_response}")
        print("Prompt sent to LLaMA2:")
        print(prompt)
        print("------------------")
        bot_response = llama_respond(prompt)
        return f"{selected_response}{bot_response}"
    else:
        print("Prompt sent to LLaMA2:")
        print(prompt)
        print("------------------")
        return llama_respond(prompt)

# ============ STEP 4: CLI Chatbot ============
if __name__ == "__main__":
    print("Welcome to the Mental Health Chatbot 💬 (type 'quit' to exit)")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["quit", "exit"]:
            print("Bot: Take care! 💙")
            break
        response = generate_response(user_input)
        print(f"Bot: {response}\n")


Loading fine-tuned RoBERTa emotion classifier...
Loading intent patterns from KB.json...
Loading Sentence-BERT model for intent classification...
Evaluating intent model self-match accuracy...
